In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
#nltk.download('wordnet')
from nltk.corpus import wordnet #wornet loaded
import re
import json
from flask import Flask,render_template, request, redirect, url_for
from flask import jsonify, current_app
from flask_cors import CORS

#Methode definition
def get_synsets(w):
    syns = wordnet.synsets(w)
    return syns

def get_definitions(syns):
    return syns.definition()

def get_label(syns):
    lemmas = syns.lemmas()
    name = lemmas[0].name()
    return name
    #lemmas = syns.lemmas()
   # name = lemmas[0].name()
    #return syns.name()

def get_class(syns):
    return syns.pos()

def get_resource(syns):
    #return syns.lexname()
    return syns.offset()  #id for a synset in wordnet db
def get_source(syns):
    #lemmas = syns.lemmas()
    #name = lemmas[0].name()
    #return name
    return syns.name()
    #return syns.lexname()
    #return syns.offset()  #id for a synset in wordnet db

def get_offset(w,text):
    offset = text.index(w)
    return offset

def stop_words_filtering(text):
    reg_exp = r"[a-zA-Z0-9]+"
 
    stop_words = set(stopwords.words('english')) #stopwords just for englisch text
    word_tokens= []
    b = []
    a = re.compile(reg_exp)
#example = "Pet food for shoes and fabric"    
        
    b = b + a.findall(text)     
    filtered_sentence =  [w for w in b if not w in stop_words] 
    filtered_sentence = [] 
    for w in b: #just filtern stopword in a text
        if w not in stop_words: 
            filtered_sentence.append(w)           
    return filtered_sentence 

#Nested dictionary
#def add_elementToRessourcedict(word, ressourcedict):
def build_ressources_candidates(word,syns,offset):
    resourcedict = {}
    resourcedict["description"] = get_definitions(syns)
    resourcedict["label"]       = get_label(syns)
    resourcedict["offset"]      = offset
    resourcedict["resource"]    = get_resource(syns)
    resourcedict["source"]      = get_source(syns)
    resourcedict["text"]        = word
    resourcedict["pos"]         = get_class(syns) #pos(part of speech oder word class)
    return resourcedict
     #resourcedict["synset_id"]   = i
    
    
    
def build_resources(word,offset):
    resources = []
    synsets = get_synsets(word)
    for syns in synsets:
        resources.append(build_ressources_candidates(word,syns,offset))
    return resources

def build_annot_candidates(word,text):
        offset = get_offset(word,text)
        candidate = {}
        candidate["offset"]                = offset
        candidate["resource_candidates"]  = build_resources(word,offset)
        candidate["text"]                  = word
        return candidate
def build_annot(text):
   # Woerter = stop_words_filtering(text)
    annotDict = {}
    annot = []
    annotDict["annotation_candidates"]   = annot
    annotDict["text"]                    = text
    Woerter = stop_words_filtering(text)
    for w in Woerter:
        annot.append(build_annot_candidates(w,text))
    return annotDict
        
        
    
    
    

In [2]:
#build_annot("pet for food")

In [ ]:
app = Flask(__name__)
CORS(app)
@app.route('/')
def index():
  return 'Server Works done!'
    
@app.route('/annotApi/<string:text>')
def annotsFunction2(text):
    annotation = build_annot(text)
    return json.dumps(annotation)

    
if __name__ == '__main__':
    with app.app_context():
   # app.debug = True
        from werkzeug.serving import run_simple
        run_simple('localhost', 4000, app)

 * Running on http://localhost:4000/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Nov/2019 19:04:00] "GET /annotApi/to%20find%20lost%20livestock HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2019 19:06:11] "GET /annotApi/the%20device%20could%20be%20used%20in%20a%20detective%20manner%20that%20is%20to%20say%20that%20it%20could%20be%20useful%20to%20predicting%20behavioral%20patterns%20of%20say%20criminal%20offenders,%20students,%20to%20perform%20research%20perhaps%20by%20tracking%20the%20physical%20lives%20of%20athletes%20or%20top%20achieving%20businessman%20or%20for%20companies%20to%20isolate%20expected%20behaviors%20of%20their%20employees%20and%20how%20they%20anticipate%20employees%20to%20move%20during%20a%20work%20day%20to%20establish%20fair%20and%20true%20standards HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2019 19:08:47] "GET /annotApi/device%20could%20be%20used HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2019 19:08:47] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/Nov/2019 19:19:50] "GET /annotApi/%